In [0]:
import pandas as pd
import numpy as np

import plotly.offline as po
import plotly.graph_objects as go
import plotly.express as px

import requests

import io

from statsmodels.tsa.seasonal import STL
from scipy import stats

from huntlib.util import benfords

from datetime import datetime

import json

import ipywidgets as widgets
from IPython.display import display

# Parameters & Setup

In [0]:
COVID_DATA_URL = 'https://data.virginia.gov/api/views/bre9-aqqr/rows.csv?accessType=DOWNLOAD'

In [0]:
w_ra_days = widgets.BoundedIntText(
    min=3,
    value=7,
    description='Rolling Average Days: '
)

w_recent_days = widgets.BoundedIntText(
    min=1,
    max=180,
    value=14,
    description="Recent Days: "
    
)

display(w_ra_days)
display(w_recent_days)

In [0]:
ROLLING_AVERAGE_DAYS = w_ra_days.value # 7
RECENT_DAYS = w_recent_days.value # 14

# Data

## Download the Most Recent Data

In [0]:
res = requests.get(COVID_DATA_URL, allow_redirects=True)

In [0]:
with open(f'cases-{datetime.now().strftime("%Y-%m-%d")}.csv', "w") as f:
    f.write(str(res.content, 'utf-8'))

## Create DataFrame

In [0]:
full_df = pd.read_csv(io.BytesIO(res.content), parse_dates=['Report Date'] )

full_df['FIPS'] = full_df['FIPS'].astype('str')
full_df['Locality'] = full_df['Locality'].astype('category')
full_df['VDH Health District'] = full_df['VDH Health District'].astype('category')

full_df.info()

## Compute Daily Deltas

In [0]:
local_dfs = list()

for locality in full_df.Locality.unique():
    ldf = full_df[full_df['Locality'] == locality].copy()
    ldf['Daily Cases'] = ldf['Total Cases'].diff().fillna(0).astype('int')
    ldf['Daily Hospitalizations'] = ldf['Hospitalizations'].diff().fillna(0).astype('int')
    ldf['Daily Deaths'] = ldf['Deaths'].diff().fillna(0).astype('int')
    local_dfs.append(ldf)
    
full_df = pd.concat(local_dfs, ignore_index=True).sort_values(by=['Report Date', 'Locality']).reindex()

full_df

# Tamper Detection

In [0]:
def check_benfords(df, columns=['Total Cases', 'Hospitalizations', 'Deaths'], threshold=0.06, p_threshold=0.99999):
    retval = True
    for col in columns:
        r, p, _ = benfords(df[col])
        if r > threshold or p <= p_threshold:
            print(f"WARNING: '{col}' did not pass the Benford's Law test ({r}, {p}).")
            retval = False 
            
    return retval

In [0]:
check_benfords(full_df)

# Graphs

## Cumulative Summary of Cases, Hospitalizations and Deaths

In [0]:
def plot_daily_cumulative_summary(df):
    cum_sum_df = df[['Report Date', 'Total Cases', 'Hospitalizations', 'Deaths']].groupby('Report Date').sum()
    
    fig = go.Figure(
        data=go.Scatter(
            x=cum_sum_df.index,
            y=cum_sum_df['Total Cases'],
            mode='lines',
            name='Total Cases'
        )
    )

    fig.add_trace(
        go.Scatter(
            x=cum_sum_df.index,
            y=cum_sum_df['Hospitalizations'],
            mode='lines',
            name='Hospitalizations'
        )
    )

    fig.add_trace(
        go.Scatter(
            x=cum_sum_df.index,
            y=cum_sum_df['Deaths'],
            mode='lines',
            name='Deaths'
        )
    )

    fig.update_layout(
        title="Virginia Cumulative Cases, Hospitalizations & Deaths"
    )

    fig.show()

In [0]:
plot_daily_cumulative_summary(full_df)

## Daily Numbers vs. Rolling Average

In [0]:
def plot_timeseries(df, column, locality=None, anomaly_threshold=3, baseline_type='ra'):

    if locality:
        local_series = df[df.Locality == locality][['Report Date', column]].groupby('Report Date').sum().sort_index()[column]
    else:
        local_series = df.groupby('Report Date').sum().sort_index()[column]
        locality = "Virginia Overall"
    
    fig = go.Figure(
        data=go.Scatter(
            x=local_series.index,
            y=local_series,
            mode='lines',
            name=column
        )
    )

    seasonal = STL(local_series, robust=True)
    
    seasonal = seasonal.fit()
    
    if baseline_type == 'trend' or baseline_type == 'trend_seasonal':
        fig.add_trace(
            go.Scatter(
                x=local_series.index,
                y=seasonal.trend + seasonal.seasonal,
                mode='lines',
                marker_color='green',
                name='Basline (Trend + Seasonal)'
            )
        )
    else:
        fig.add_trace(
            go.Scatter(
                x=local_series.index,
                y=local_series.rolling(ROLLING_AVERAGE_DAYS).mean(),
                mode='lines',
                marker_color='green',
                name="Baseline (Rolling Average)"
            )
        )
    
    anomalies = seasonal.resid[abs(seasonal.resid - seasonal.resid.mean()) >= (anomaly_threshold * seasonal.resid.std())]

    fig.add_trace(
        go.Scatter(
            x=anomalies.index,
            y=local_series.loc[anomalies.index],
            mode='markers',
            marker_symbol='x',
            marker_color='red',
            marker = dict(
              size=10  
            ),
            name='Anomalies'
        )
    )
    
    fig.update_layout(
        title=f"COVID-19 {column} vs {ROLLING_AVERAGE_DAYS} Day Average: {locality}"
    )

    fig.show()


### Virginia Overall

In [0]:
plot_timeseries(full_df, column='Daily Cases')
plot_timeseries(full_df, column='Daily Hospitalizations')
plot_timeseries(full_df, column='Daily Deaths')

In [0]:
widget_locality_chooser = widgets.Dropdown(
    options=full_df.Locality.unique(),
    value='James City',
    description="Locality: "
)

@widgets.interact(locality=widget_locality_chooser)
def make_locality_plots(locality):
    plot_timeseries(full_df, column='Daily Cases', locality=locality)
    plot_timeseries(full_df, column='Daily Hospitalizations', locality=locality)
    plot_timeseries(full_df, column='Daily Deaths', locality=locality)

## Activity Heatmaps by County

In [0]:
with open('geojson-counties-fips.json', 'r') as f:
    counties = f.read()
    
counties = json.loads(counties)

In [0]:
start_time = full_df[-1:]['Report Date'] - pd.DateOffset(RECENT_DAYS, 'D')

recent_df = full_df[full_df['Report Date'] >= start_time.iloc[0]]

recent_df = recent_df.drop(['Total Cases', 'Hospitalizations', 'Deaths'], axis='columns')

recent_df = recent_df.rename(
    {
        'Daily Cases': 'Cases',
        'Daily Hospitalizations': 'Hospitalizations',
        'Daily Deaths': 'Deaths'
    },
    axis='columns'
)

recent_df = recent_df.groupby(['FIPS', 'Locality']).sum().reset_index()
recent_df.head()

In [0]:
def plot_heatmap(df, geojson, column=None, locations='FIPS', height=1000, width=None, color_scale='viridis'):
    fig = px.choropleth(
        df,
        geojson=geojson,
        locations=locations,
        color=column,
        scope='usa',
        hover_data=['Locality', 'FIPS', column],
        height=height,
        width=width,
        color_continuous_scale=color_scale,
        title=f"{column} in Last {RECENT_DAYS} Days by County"
    )

    fig.update_geos(fitbounds='locations')
    
    fig.show()

In [0]:
plot_heatmap(recent_df, counties, column='Cases')

In [0]:
plot_heatmap(recent_df, counties, column='Hospitalizations')

In [0]:
plot_heatmap(recent_df, counties, column='Deaths')